In [1]:
import numpy as np
import optuna

import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

In [ ]:
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [2]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-10-14 10:23:09,133] A new study created in memory with name: no-name-025b0f37-8f1a-40c7-8b93-bc809fc8ffc7
[I 2022-10-14 10:23:09,291] Trial 0 finished with value: 0.9790209790209791 and parameters: {'lambda_l1': 1.4691018903197828e-05, 'lambda_l2': 3.917276481700524e-06, 'num_leaves': 140, 'feature_fraction': 0.7466046532207959, 'bagging_fraction': 0.5884751642226065, 'bagging_freq': 5, 'min_child_samples': 44}. Best is trial 0 with value: 0.9790209790209791.
[I 2022-10-14 10:23:09,868] Trial 1 finished with value: 0.972027972027972 and parameters: {'lambda_l1': 1.071028212100454e-08, 'lambda_l2': 1.3646933180578588e-08, 'num_leaves': 209, 'feature_fraction': 0.6166815960472101, 'bagging_fraction': 0.8396599039016893, 'bagging_freq': 7, 'min_child_samples': 14}. Best is trial 0 with value: 0.9790209790209791.
[I 2022-10-14 10:23:09,975] Trial 2 finished with value: 0.958041958041958 and parameters: {'lambda_l1': 0.58235845946282, 'lambda_l2': 6.47998442746459, 'num_leaves': 3, 

Number of finished trials: 100
Best trial:
  Value: 0.993006993006993
  Params: 
    lambda_l1: 4.2385086800987535e-05
    lambda_l2: 1.5319811585661526e-07
    num_leaves: 207
    feature_fraction: 0.6638969257390502
    bagging_fraction: 0.5689351738182228
    bagging_freq: 6
    min_child_samples: 41
